# Chatbot with Message history


In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="Gemma2-9b-It")

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

response = llm.invoke(
    [
        HumanMessage(
            content="Hi, my name is Abhinav. I want to be a Data Scientist one day."
        )
    ]
)

print(response.content)

Hi Abhinav, that's great! Data Science is a fascinating and rewarding field.  

What interests you most about data science? Do you have any experience with programming, statistics, or mathematics already? 

I can help you explore the path to becoming a data scientist. We can talk about:

* **Essential skills:** What skills do you need to learn?
* **Learning resources:** Where can you find good courses, books, and online tutorials?
* **Projects:** What kind of projects can you build to showcase your skills?
* **Career paths:** What are the different roles available in data science? 

Let me know what you'd like to discuss first!



In [6]:
# No memory
llm.invoke([HumanMessage(content="Do you know what is my name and my goal?")])

AIMessage(content="As an AI, I have no memory of past conversations and no access to personal information about you. So I don't know your name or your goal.\n\nIf you'd like to tell me your name and goal, I'd be happy to listen! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 20, 'total_tokens': 78, 'completion_time': 0.105454545, 'prompt_time': 7.549e-05, 'queue_time': 0.019630698, 'total_time': 0.105530035}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ecbe5f6d-1507-4919-94b9-ba040f809307-0', usage_metadata={'input_tokens': 20, 'output_tokens': 58, 'total_tokens': 78})

In [7]:
# With memory
llm.invoke(
    [
        HumanMessage(
            content="Hi, my name is Abhinav. I want to be a Data Scientist one day."
        ),
        AIMessage(content=response.content),
        HumanMessage(content="Do you know what is my name and my goal?"),
    ]
)

AIMessage(content="Yes, I do!  You introduced yourself as Abhinav, and you said you want to be a Data Scientist one day.  \n\nIs there anything specific you'd like to know or discuss about that goal right now?  Maybe we could brainstorm some steps you could take? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 193, 'total_tokens': 257, 'completion_time': 0.116363636, 'prompt_time': 0.007193791, 'queue_time': 0.020257216, 'total_time': 0.123557427}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0cc6b551-9ddb-481d-b719-793c69a43196-0', usage_metadata={'input_tokens': 193, 'output_tokens': 64, 'total_tokens': 257})

## Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.


In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create a store to store message histories based on session id
store = {}


# Function to retrieve the message history based on session id
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# LLM model that can work with message histories
llm_with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [11]:
# Create a session config
config = {"configurable": {"session_id": "chat1"}}

In [ ]:
response = llm_with_message_history.invoke(
    [HumanMessage(content="My name is Abhinav, what is your name?")], config=config
)
print(response.content)

Hi Abhinav, my name is Gemma.  

It's nice to meet you! 😄 




In [15]:
response = llm_with_message_history.invoke(
    [HumanMessage(content="What is my name?")], config=config
)
print(response.content)

Your name is Abhinav.  I remember! 😊  

Is there anything else I can help you with?



In [17]:
# Changing the session id
config2 = {"configurable": {"session_id": "chat2"}}

response = llm_with_message_history.invoke(
    [HumanMessage(content="What is my name?")], config=config2
)
print(response.content)

As an AI, I have no memory of past conversations and no access to personal information about you, including your name. If you'd like to tell me your name, I'd be happy to use it!



## Prompt Template


In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Prompt template with single variable Message Placeholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

In [19]:
chain.invoke({"messages": ["My name is Abhinav"]})

AIMessage(content="Hi Abhinav! It's nice to meet you. \n\nI'm ready to answer your questions to the best of my ability. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 32, 'total_tokens': 72, 'completion_time': 0.072727273, 'prompt_time': 0.00030438, 'queue_time': 0.020779396, 'total_time': 0.073031653}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-51092b7b-35fe-4485-be2e-f436420cf088-0', usage_metadata={'input_tokens': 32, 'output_tokens': 40, 'total_tokens': 72})

In [24]:
# Along with message history
llm_with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "chat3"}}

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi my name is Abhinav. What is your name?")],
    },
    config=config,
)

print(response.content)

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What is my name?")],
    },
    config=config,
)

print(response.content)

Hi Abhinav, I am Gemma!  It's nice to meet you. 😊 

What can I help you with today?  

Your name is Abhinav.  😊 

Is there anything else I can help you with?



In [30]:
# Prompt template with multiple variables
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the questions in {language}"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

In [31]:
chain.invoke({"messages": ["My name is Abhinav"], "language": "hindi"})

AIMessage(content='Namaste Abhinav! \n\nआपके नाम सुनकर अच्छा लगा। \n\nमुझे आपसे कुछ सवाल पूछने का मौका मिलने पर मुझे खुशी हो रही है। आप क्या जानना चाहेंगे? 😊\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 20, 'total_tokens': 73, 'completion_time': 0.096363636, 'prompt_time': 7.461e-05, 'queue_time': 0.020157298, 'total_time': 0.096438246}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-eee003d5-716b-422d-b518-9485343e524f-0', usage_metadata={'input_tokens': 20, 'output_tokens': 53, 'total_tokens': 73})

In [33]:
llm_with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat4"}}

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi my name is Abhinav. What is your name?")],
        "language": "hindi",
    },
    config=config,
)

print(response.content)

response = llm_with_message_history.invoke(
    {"messages": [HumanMessage(content="What is my name?")], "language": "hindi"},
    config=config,
)

print(response.content)

नमस्ते अभिनव, मेरा नाम है जेन।  

तुम्हारा नाम अभिनव है।  



## Trimming message history

If the conversation history is left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM model. Therefore it is important to trim the messages we are passing to the model.

`trim_messages` is a helper which is used to reduce the number of messages we are sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always want to keep the system message and whether to allow partial messages


In [38]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [47]:
# Chaining with trimmer
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | llm
)

response = chain.invoke({
    "messages": messages + [HumanMessage(content="What ice cream do I like?")],
    "language": "Hindi"
})

print(response.content)

response = chain.invoke({
    "messages": messages + [HumanMessage(content="What math problem did I ask for?")],
    "language": "Hindi"
})

print(response.content)

मुझे नहीं पता कि आपको किस तरह का आइसक्रीम पसंद है! आपका पसंदीदा आइसक्रीम क्या है?  🍦 🍨

आपने मुझसे 2 + 2 का उत्तर पूछा था।  



In [102]:
# Wrapping with message history runnable
chain = prompt | llm

def get_trimmed_message_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
    
llm_with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_trimmed_message_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "chat5"}}

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi, my name is Abhinav")],
        "language": "English",
    },
    config=config,
)

print(response.content)

response = llm_with_message_history.invoke(
    {"messages": [HumanMessage(content="What is my name?")], "language": "English"},
    config=config,
)

print(response.content)

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="I like vanilla ice cream")],
        "language": "English",
    },
    config=config,
)

print(response.content)

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What ice cream do I like?")],
        "language": "English",
    },
    config=config,
)

print(response.content)

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Data Science is my favorite subject")],
        "language": "English",
    },
    config=config,
)

print(response.content)

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What is my favorite subject?")],
        "language": "English",
    },
    config=config,
)

print(response.content)

response = llm_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What is my name?")],
        "language": "English",
    },
    config=config,
)

print(response.content)

Hi Abhinav,

It's nice to meet you!  

What can I do for you today? Do you have any questions I can answer or tasks I can help you with? 😊 


Your name is Abhinav.  

I remember that you told me when you first introduced yourself! 😊  Is there anything else I can help you with?

That's great! Vanilla ice cream is a classic for a reason.  

Do you have a favorite way to enjoy it? With sprinkles, in a sundae, or just plain? 🍦

You like vanilla ice cream!  🍦  You told me so earlier. 😊  



What's your favorite topping for it? 

That's awesome, Abhinav! Data Science is a really fascinating field. What do you enjoy most about it? 

Is there a particular area of data science that interests you the most?  🤔 


Your favorite subject is Data Science!  You told me just now. 😊  

Are you learning a lot in your Data Science classes?  





Your name is Abhinav.  

I remember!  Is there anything else I can help you with today? 😊 

